# Landslide Hazard Analysis Using SAR
Intro to the use case/methods used

Search for SLC images over Haiti in a defined time period

In [ ]:
import os
from datetime import datetime

start_time = datetime.strptime('2021-05-01T23:59', '%Y-%m-%dT%H:%M')
end_time = datetime.strptime('2021-08-16T23:59', '%Y-%m-%dT%H:%M')
event_time = datetime.strptime('2021-08-14T00:00', '%Y-%m-%dT%H:%M')

import asf_search
scenes_to_submit = []
wkt = 'POLYGON((-74.6 18.0,-73.0 18.0,-73.0 18.8,-74.6 18.8, -74.6 18.0))'
results = asf_search.geo_search(platform=[asf_search.PLATFORM.SENTINEL1], intersectsWith=wkt, processingLevel='SLC', start=start_time, end=end_time)
[scenes_to_submit.append(result.properties['sceneName']) for result in results]
print(f'There are {len(scenes_to_submit)} scenes in the AOI between {start_time} and {end_time}')

Request HyP3 processing of Haiti data for two years

In [ ]:
import hyp3_sdk as sdk

print(f'Submitting {len(scenes_to_submit)} jobs')
hyp3 = sdk.HyP3()
rtc_jobs = sdk.Batch()
for scene in scenes_to_submit:
    rtc_jobs += hyp3.submit_rtc_job(granule=scene, name='IGARSS-HAITI')

In [ ]:
data_dir_path = 'haiti_rtcs'
rtc_jobs = hyp3.watch(rtc_jobs)
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
file_list = succeeded_jobs.download_files(location=data_dir_path)

Download the staged version of the data

In [ ]:
import hyp3_sdk as sdk
hyp3 = sdk.HyP3()

jobs = hyp3.find_jobs(user_id='jrsmale', name='IGARSS-HAITI')

data_dir_path = 'haiti_rtcs'
months_before = datetime(2021,7,14)
dates = [datetime.strptime(job.to_dict()['job_parameters']['granules'][0][33:41], '%Y%m%d') for job in jobs]
indexes = [i for i,date in enumerate(dates) if date >= months_before]
desired_jobs = sdk.Batch([jobs[i] for i in indexes])
if not os.path.isdir(data_dir_path):
    os.mkdir(data_dir_path)
download_paths = desired_jobs.download_files(location=data_dir_path)

Crop all images to same extent (e.g. InSAR Notebook)

In [ ]:
from pathlib import Path
from typing import List, Union
from osgeo import gdal

def clip_hyp3_products_to_same_extent(data_dir: List[Union[str, Path]]) -> List[str]:
    """Clip all GeoTIFF hyp3 products to a specified extent
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        extent:
            a list of the upper-left x, upper-left y, lower-right x, and lower-right y corner coordinates of desired extent.
    Returns: None
    """
    for file in data_dir.glob(f'**/*VV.tif'):
        dst_file = Path(str(file.absolute()).replace(f'VV.tif', f'VV_clipped.tif'))
        gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=extent, projWinSRS="EPSG:4326")

In [ ]:
extent = [-74.6, 18.8, -73.0, 18.0]
clip_hyp3_products_to_same_extent(download_paths, extent)

Load the geotiffs into Xarray with datetime stamps for each image

In [ ]:
def extract_tif_names(scene_dir_path):
    scene_path_listed = os.listdir(scene_dir_path)
    scene_files_vv = [fname for fname in scene_path_listed if fname.endswith('_VV_clipped.tif')]
    return scene_files_vv

scenes_listed = os.listdir(data_dir_path)
fpaths_vv = []

for element in range(len(scenes_listed)):
    print(f'{data_dir_path}/{scenes_listed[element]}')
    try:
        good_files = extract_tif_names(f'{data_dir_path}/{scenes_listed[element]}')
        fpaths_vv.append(f'{data_dir_path}/{scenes_listed[element]}/{good_files[0]}')
    except:
        print(f'Couldnt use {scenes_listed[element]}. Skipping.')
        continue

Pre-temporal averaging, create time series plot of single pixel (or group of pixels)

In [ ]:
import xarray as xr

def preprocess(da_orig, file_type: str='vv'):
    '''function that should return an xarray object with time dimension and associated metadata given a path to a single RTC scene, if its dualpol will have multiple bands, currently just as 2 data arrays but could merge.
    goal would be to apply this a list of directories for different RTC products, return cube along time dimension - I think?
    - for concatenating, would need to check footprints and only take products with the same footprint, or subset them all to a common AOI? '''
    da = da_orig.copy()
    da = da.rename({'band_data': file_type}).squeeze()
    fname = os.path.basename(da_orig['band_data'].encoding['source'])
    date = datetime.strptime(fname[7:22], '%Y%m%dT%H%M%S')
    da = da.assign_coords({'acq_date': date})
    da = da.expand_dims('acq_date')
    da = da.drop_duplicates(dim=['x', 'y'])

    return da


vv = xr.open_mfdataset(paths = fpaths_vv, preprocess = preprocess, chunks = 'auto', engine='rasterio', data_vars='minimal', coords='minimal', concat_dim='acq_date', combine='nested', parallel=True)

Create a temporal average for both pre and post EQ

In [ ]:
import numpy as np

date_bins = [start_time, event_time, end_time]
date_bin_labels = ["preevent", "postevent"]

vv = vv.groupby_bins("acq_date", date_bins, labels=date_bin_labels)
mean_vv = vv.mean(dim='acq_date')

Perform a log difference of the two resulting images.

In [ ]:
log_diff = 10*np.log10(mean_vv['vv'][0]/mean_vv['vv'][1])

import matplotlib.pyplot as plt

plt.matshow(log_diff)
plt.colorbar()
plt.show()

Discuss options for thresholding the difference image

Use the COP30 DEM to remove change detections in areas with slopes <5 degrees use gdaldem

In [ ]:
from dem_stitcher import stitch_dem


dem_file = data_dir_path / 'DEM.tif'
bounds = [-74.6, 18.0, -73.0, 18.8]
X, p = stitch_dem(bounds,
                  dem_name='glo_30',  # Global Copernicus 30 meter resolution DEM
                  dst_ellipsoidal_height=False,
                  dst_area_or_point='Point')

import rasterio

with rasterio.open(dem_file, 'w', **p) as ds:
   ds.write(X, 1)
   ds.update_tags(AREA_OR_POINT='Point')

process_dem_file = data_dir_path / 'DEM_processed.tif'
gdal.DEMProcessing(destName=str(process_dem_file), srcDS=str(dem_file), processing="slope", format="Gtiff", slopeFormat="degree")

height, width = np.shape(log_diff)
gdal.Warp(str(process_dem_file), str(process_dem_file), dstSRS='EPSG:4326',
                  outputBounds=bounds, width=width, height=height, resampleAlg='nearest', format='GTiff')

dem = gdal.Open(str(process_dem_file), gdal.GA_ReadOnly).ReadAsArray()
dem_mask = dem > 5

log_diff = log_diff.where(dem_mask)

 Plot a histogram for the difference image (might have to subset histogram source to area with equal amounts of landslides and non-landslides).

In [ ]:
import matplotlib.pyplot as plt

plt.hist(log_diff)
plt.show()

Create a figure to interactively view the data and set a threshold using matplotlib

In [ ]:
plt.matshow(log_diff.where(dem_mask))
plt.show()